In [1]:
import os
import csv
from collections import namedtuple
from datetime import datetime

import openpyxl

In [2]:
# Availables!

sourcepath = 'data/databasetables'
available_file = 'prod_etd_available_database.xlsx'
available_wb = openpyxl.load_workbook(os.path.join(sourcepath, available_file))

In [3]:
# these items' files were chpt-by-chpt pdfs, joined into one pdf post upload.
# we must be careful to ingest to Digital Commons the joined files instead of the split files.

joined_postupload = ('etd-06182004-122626', 'etd-09012004-114224', 'etd-0327102-091522', 'etd-0707103-142120',
                    'etd-0710102-054039', 'etd-0409103-184148', 'etd-04152004-142117', 'etd-0830102-145811',
                    'etd-0903103-141852', )

In [4]:
# list all the sheets within the xsls workbook

available_sheets = [sheet for sheet in available_wb.get_sheet_names()]
print(available_sheets)

['prod_etd_available all tables', 'keyword_by_urn table', 'filename_by_urn table', 'etd_main table', 'advisor_by_urn table']


In [5]:
# Submitteds!

sourcepath = 'data/databasetables'
submitted_file = 'prod_etd_submitted_database.xlsx'
submitted_wb = openpyxl.load_workbook(os.path.join(sourcepath, submitted_file))

In [6]:
submitted_sheets = [sheet for sheet in submitted_wb.get_sheet_names()]
print(submitted_sheets)

['prod_etd_submitted all tables', 'keyword_by_urn table', 'filename_by_urn table', 'etd_main table', 'advisor_by_urn table']


In [7]:
# Withhelds!

sourcepath = 'data/databasetables'
withheld_file = 'prod_etd_submitted_database.xlsx'
withheld_wb = openpyxl.load_workbook(os.path.join(sourcepath, withheld_file))

In [8]:
withheld_sheets = [sheet for sheet in withheld_wb.get_sheet_names()]
print(withheld_sheets)

['prod_etd_submitted all tables', 'keyword_by_urn table', 'filename_by_urn table', 'etd_main table', 'advisor_by_urn table']


Ok, we've read the Available, Withheld, and Submitted xlsx files into memory.  Everything else is fast now.


In [9]:
def parse_main_sheet():
    """ returns a dictionary in form of:
    {urn: NamedTuple
     urn: NamedTuple
    }
    NamedTuple is expected to have attributes: (urn first_name middle_name last_name suffix author_email 
                                                publish_email degree department dtype title abstract availability
                                                availability_description copyright_statement ddate sdate adate
                                                cdate rdate pid url notice notice_response timestamp
                                                survey_completed)
                                            or: (urn first_name middle_name last_name suffix author_email
                                                publish_email degree department dtype title abstract availability
                                                availability_description copyright_statement ddate sdate adate
                                                cdate rdate pid url notices timestamp)
    )
    """
    main_dict = dict()
    for wb in (available_wb, submitted_wb, withheld_wb):
        current_sheet = wb.get_sheet_by_name('etd_main table')
        for num, row in enumerate(current_sheet.iter_rows()):
            if num == 0:
                headers = (i.value for i in row)
                MainSheet = namedtuple('MainSheet', headers)
                continue
            values = (i.value for i in row)
            item = MainSheet(*values)
            main_dict[item.urn] = item
    return main_dict

In [10]:
def parse_filename_sheet():
    """returns a dictionary in form of:
    {urn: {filename: (path, size, available, description, page_count, timestamp),
           filename: (path, size, available, description, page_count, timestamp),}
     urn: {filename: (path, size, available, description, page_count, timestamp),}
    """
    filenames_sheet = dict()
    for wb in (available_wb, submitted_wb, withheld_wb):
        current_sheet = wb.get_sheet_by_name('filename_by_urn table')
        for num, row in enumerate(current_sheet.iter_rows()):
            if num == 0:
                continue
            [urn, filename, path, size, available, description, page_count, timestamp] = [i.value for i in row]
            file_dict = {filename: (path, size, available, description, page_count, timestamp)}
            if urn not in filenames_sheet:
                filenames_sheet[urn] = file_dict
            else:
                row_timestamp = datetime.strptime(timestamp, "%Y-%m-%d %H:%M:%S")
                if filename in filenames_sheet[urn] and filenames_sheet[urn][filename][3] and filenames_sheet[urn][filename][3] != "NULL":
                    last_timestamp = datetime.strptime(filenames_sheet[urn][filename][3], "%Y-%m-%d %H:%M:%S")
                    if row_timestamp > last_timestamp:
                        filenames_sheet[urn][filename] = (path, size, available, description, page_count, timestamp)
                else:
                    filenames_sheet[urn][filename] = (path, size, available, description, page_count, timestamp)
    return filenames_sheet

In [11]:
def parse_keyword_sheet():
    """ returns a dictionary in form of:
    {urn: [(keyword, timestamp),
           (keyword, timestamp),
           ]}
    """
    keywords_sheet = dict()
    for wb in (available_wb, submitted_wb, withheld_wb):
        current_sheet = wb.get_sheet_by_name('keyword_by_urn table')
        for num, row in enumerate(current_sheet.iter_rows()):
            if num == 0:
                continue
            [keyword, urn, timestamp] = [i.value for i in row]
            if urn not in keywords_sheet:
                keywords_sheet[urn] = [(keyword, timestamp), ]
            else:
                keywords_sheet[urn].append((keyword, timestamp))
    return keywords_sheet

In [12]:
def parse_advisors_sheet():
    """ returns a dictionary in form of:
        {urn: {advisor: (advisor_title, advisor_email, approval, timestamp),
               advisor: (advisor_title, advisor_email, approval, timestamp),}
         urn: {advisor: (advisor_title, advisor_email, approval, timestamp),}
         }
    """
    advisors_sheet = dict()
    for wb in (available_wb, submitted_wb, withheld_wb):
        current_sheet = wb.get_sheet_by_name('advisor_by_urn table')
        for num, row in enumerate(current_sheet.iter_rows()):
            if num == 0:
                continue
            [urn, advisor_name, advisor_title, advisor_email, approval, timestamp] = [i.value for i in row]
            advisor_dict = {advisor_name: (advisor_title, advisor_email, approval, timestamp)}
            if urn not in advisors_sheet:
                advisors_sheet[urn] = advisor_dict
            else:
                row_timestamp = datetime.strptime(timestamp, "%Y-%m-%d %H:%M:%S")
                if advisor_name in advisors_sheet[urn]:
                    last_timestamp = datetime.strptime(advisors_sheet[urn][advisor_name][3], "%Y-%m-%d %H:%M:%S")
                    if row_timestamp > last_timestamp:
                        advisors_sheet[urn][advisor_name] = (advisor_title, advisor_email, approval, timestamp)
                else:
                    advisors_sheet[urn][advisor_name] = (advisor_title, advisor_email, approval, timestamp)
    return advisors_sheet

In [13]:
def parse_catalog_sheet():
    catalog_sheet = dict()
    sourcepath = 'data/Catalogtables'
    sourcefile = 'CatalogETDSelectMetadata.csv'
    with open(os.path.join(sourcepath, sourcefile)) as csvfile:
        csvreader = csv.reader(csvfile, delimiter=',')
        count = 0
        for num, row in enumerate(csvreader):
            if num == 0:
                continue
            # row = [i.strip('/').strip(']').strip('[').replace('])', '').replace('\n', '') for i in row]
            Title, Subtitle, AuthorFromTitleField, Author, SeriesDate, PubDate, URL = row
            urn = [i for i in URL.split('/') if 'etd-' in i]
            urn = urn[0]
            if not urn:
                print('No urn for URL:', URL)
            if urn not in catalog_sheet:
                catalog_sheet[urn] = [(Title, Subtitle, AuthorFromTitleField, Author, SeriesDate, PubDate, URL), ]
            else:
                catalog_sheet[urn].append((Title, Subtitle, AuthorFromTitleField, Author, SeriesDate, PubDate, URL))
                count += 1
    output_srt = ''
    for k, item in catalog_sheet.items():
        if len(item) > 1:
            output_srt += '\n{}\n'.format(k)
            for thing in item:
                output_srt += '\n'.join(thing)
                output_srt += '\n'
            output_srt += '\n\n'
    return catalog_sheet

In [14]:
# we just made plans for how to convert the openpyxls datastructure into python datastructures.  Now let's do it.
# take note that each sheet gets a slightly different datastructure, as they describe different data.

# print an example of each, for later reference.

main_sheet = parse_main_sheet()
catalog_sheet = parse_catalog_sheet()
filenames_sheet = parse_filename_sheet()
keywords_sheet = parse_keyword_sheet()
advisors_sheet = parse_advisors_sheet()

print("Main example:", main_sheet['etd-0821101-100809'], "\n")
print("Catalog example:", catalog_sheet['etd-0821101-100809'], "\n")
print("Filenames example:", filenames_sheet['etd-0821101-100809'], "\n")
print("Keywords example:", keywords_sheet['etd-0821101-100809'], "\n")
print("Advisors example:", advisors_sheet['etd-0821101-100809'], "\n")

Main example: MainSheet(urn='etd-0821101-100809', first_name='Wade', middle_name='R.', last_name='Hampton', suffix=None, author_email='waderhampton@yahoo.com', publish_email='NO', degree='MS', department='Agricultural Economics and Agribusiness', dtype='thesis', title='Trade Flows and Marketing Practices of Louisiana and Gulf States Nurseries', abstract='The markets facing nursery producers have changed dramatically in the past decade. These changes in nursery markets have outdated previous research. New research into nursery marketing will assist nursery producers in making marketing decisions. Nursery producers can market their plants to five different marketing channels: Mass-merchandisers, garden centers, other retailers, landscapers and re-wholesalers. This study described the 1998 Louisiana nursery industry, analyzed nursery market changes over the past decade in Louisiana and the Southeast and analyzed characteristics of Louisiana and Southeastern nurseries to estimate marketing

In [15]:
# investigate the advisors sheet.  What are the permutations of the advisor types in all the etd records?

urn_advisortitles = dict()
for urn, chunk in advisors_sheet.items():
    for advisor, thing in chunk.items():
        advisor_title, advisor_email, approval, timestamp = thing
        if urn in urn_advisortitles:
            urn_advisortitles[urn].append(advisor_title)
        else:
            urn_advisortitles[urn] = [advisor_title,]

a_set = set()
for k, v in urn_advisortitles.items():
    for i in v:
        a_set.add(i)
print(a_set)

advisors_permutations = set()

for k, v in urn_advisortitles.items():
    this_permutation = (v.count('Committee Chair'),
                        v.count('Committee Co-Chair'),
                        v.count('Committee Member'),
                        v.count("Dean's Representative"),
                       )
    advisors_permutations.add(this_permutation)
for i in advisors_permutations:
    print(i)

{'Committee Chair', 'Committee Member', 'Committee Co-Chair', "Dean's Representative"}
(1, 0, 3, 2)
(1, 0, 4, 2)
(3, 0, 2, 1)
(1, 2, 3, 0)
(0, 2, 2, 1)
(1, 0, 5, 1)
(1, 1, 1, 1)
(0, 0, 3, 1)
(1, 2, 0, 0)
(2, 0, 1, 0)
(0, 2, 5, 0)
(0, 2, 4, 0)
(1, 0, 7, 1)
(1, 1, 3, 1)
(2, 1, 0, 1)
(2, 0, 2, 0)
(1, 0, 1, 1)
(0, 1, 3, 1)
(1, 1, 2, 2)
(0, 2, 1, 0)
(1, 0, 2, 3)
(0, 2, 0, 0)
(1, 0, 3, 1)
(2, 0, 4, 0)
(1, 1, 4, 1)
(1, 2, 2, 1)
(0, 2, 3, 0)
(1, 0, 4, 0)
(5, 0, 0, 1)
(0, 2, 2, 0)
(1, 0, 5, 0)
(0, 0, 4, 1)
(0, 3, 2, 1)
(0, 2, 5, 1)
(1, 0, 6, 0)
(0, 0, 2, 1)
(2, 0, 3, 1)
(1, 0, 0, 0)
(1, 0, 1, 0)
(1, 1, 2, 1)
(0, 0, 0, 1)
(0, 2, 1, 1)
(1, 0, 2, 0)
(1, 1, 5, 0)
(1, 0, 3, 0)
(1, 1, 4, 0)
(0, 0, 6, 0)
(0, 2, 3, 1)
(1, 0, 3, 3)
(1, 0, 4, 1)
(0, 0, 5, 0)
(0, 0, 4, 0)
(0, 3, 1, 1)
(1, 0, 6, 1)
(1, 1, 1, 0)
(0, 0, 3, 0)
(3, 0, 0, 0)
(0, 1, 4, 1)
(1, 0, 0, 4)
(1, 1, 0, 0)
(1, 2, 1, 1)
(0, 2, 4, 1)
(0, 1, 2, 1)
(1, 1, 3, 0)
(1, 1, 2, 0)
(2, 0, 2, 1)
(1, 0, 2, 1)
(1, 0, 2, 2)


In [16]:
# locates urns without the expected file

# also locates items labeled 'available' in one place & 'withheld' in another
#   (irrelevant since we're treating these the same)

sames = dict()
for urn, bunch in filenames_sheet.items():
    for filename, (path, size, available, description, page_count, timestamp) in bunch.items():
        if urn in sames:
            if sames[urn] != available:
                print('there should be one {}'.format(urn))
        else:
            sames[urn] = available

# for urn, itemnamedtuple in main_sheet.items():
#     if urn not in sames:
#         print('{} wasnt in filenames sheet'.format(urn))
#     elif sames[urn] != itemnamedtuple.availability:
#         print(urn, sames[urn], itemnamedtuple.availability)
        
# etd-08082016-164729 wasnt in filenames sheet   --  no pdf submitted yet 
# etd-06092008-192351 unrestricted withheld      --  listed in available & withheld databases
# etd-06062010-192030 wasnt in filenames sheet   --  no pdf submitted yet


In [17]:
# files without proper file extension

for etd, filenames in filenames_sheet.items():
    for filename, _ in filenames.items():
        if filename[-4] != "." and filename[-4:] not in ("docx", "r.gz"):
            print(etd, filename)

In [18]:
# Legacy school names

schools_etds = dict()

for urn, itemnamedtuple in main_sheet.items():
    if itemnamedtuple.department in schools_etds:
        schools_etds[itemnamedtuple.department].append(urn)
    else:
        schools_etds[itemnamedtuple.department] = [urn, ]

# for school, urns in schools_etds.items():
#     print(school)

In [19]:
# Files split into parts

split_files = dict()

for urn, bunch in filenames_sheet.items():
    for filename, _ in bunch.items():
        if urn not in split_files:
            split_files[urn] = [filename,]
        else:
            split_files[urn].append(filename)

for urn, filelist in split_files.items():
    split = False
    for i in filelist:
        if "chap" in i.lower():
            split = True
    if len(filelist) > 1 and split == True:
        print(urn, '\n', filelist, '\n')

In [20]:
# Scripts for scraping binaries from the webpage to our local drive

import urllib.request

def retrieve_binary(url, filename):
    with urllib.request.urlopen(url) as response:
        return response.read()

In [21]:
def write_binary_to_file(binary, folder, filename):
    os.makedirs(folder, exist_ok=True)
    filepath = os.path.join(folder, filename)
    with open(filepath, 'bw') as f:
        f.write(binary)

In [22]:
# Scrape all the binaries

# import os

# didnt_grab = []
# target_dir = '/home/francis/Desktop/ETDbinaries/'

# for urn, filebunch in filenames_sheet.items():
#     local_dir = os.path.join(target_dir, urn)
#     os.makedirs(local_dir, exist_ok=True)
#     local_files = os.listdir(local_dir)
#     for filename, (path, size, available, description, page_count, timestamp) in filebunch.items():
#         if filename in local_files:
#             pass
#         else:
#             url = 'http://etd.lsu.edu/{}/{}'.format("/".join(path.split('/')[3:]), filename)
#             try:
#                 binary = retrieve_binary(url, filename)
#                 write_binary_to_file(binary, local_dir, filename)
#             except:
#                 didnt_grab.append((urn, filename))
#                 print(urn, filename)

In [23]:
# Helper script that prints all the info related to a urn.

def show_all_info(urn):
    print("Main example:", main_sheet[urn], "\n")
    print("Filenames example:", filenames_sheet[urn], "\n")
    print("Keywords example:", keywords_sheet[urn], "\n")
    print("Advisors example:", advisors_sheet[urn], "\n")
    print("Catalog example:", catalog_sheet[urn], "\n")

In [24]:
# prints the urn, file location, and filename for urns of which we couldn't grab the binaries

# for urn, filename in didnt_grab:
#     print(urn, filenames_sheet[urn][filename][0], filename)

In [25]:
# show_all_info('etd-09012004-114224')